In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import accuracy_score, \
    precision_score, \
    recall_score
from sklearn.metrics import f1_score
from sklearn.datasets import make_blobs
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
import math
import time
from collections import defaultdict
from typing import Optional
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('cirrhosis.csv')

In [ ]:
training_data, testing_data = train_test_split(df, test_size=0.2, random_state=25, shuffle=True)

0      False
1      False
2      False
3      False
4       True
       ...  
743     True
744     True
745     True
746     True
747     True
Name: 4, Length: 748, dtype: bool

In [ ]:
# training_data.to_csv('MTrain.csv', index=False)
# testing_data.to_csv('MTest.csv', index=False)

In [ ]:
df_train = pd.read_csv('dermatologyTrain.csv', 
                       header=None
                       )
df_test = pd.read_csv('dermatologyTest.csv', 
                      header=None
                      )

train_size = len(df_train)

train_X = df_train.iloc[:180, :-1]
train_y = df_train.iloc[:180,-1]
test_X = df_test.iloc[:62, :-1]
test_y = df_test.iloc[:62,-1]

# train_X = df_train.iloc[:, 1:]
# train_y = df_train.iloc[:,0]
# test_X = df_test.iloc[:, 1:]
# test_y = df_test.iloc[:,0]


# train_y = df_train['Status']
# train_X = df_train.drop('Status', axis=1)

# test_y = df_test['Status']
# test_X = df_test.drop('Status', axis=1)


In [ ]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 34 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       287 non-null    int64
 1   1       287 non-null    int64
 2   2       287 non-null    int64
 3   3       287 non-null    int64
 4   4       287 non-null    int64
 5   5       287 non-null    int64
 6   6       287 non-null    int64
 7   7       287 non-null    int64
 8   8       287 non-null    int64
 9   9       287 non-null    int64
 10  10      287 non-null    int64
 11  11      287 non-null    int64
 12  12      287 non-null    int64
 13  13      287 non-null    int64
 14  14      287 non-null    int64
 15  15      287 non-null    int64
 16  16      287 non-null    int64
 17  17      287 non-null    int64
 18  18      287 non-null    int64
 19  19      287 non-null    int64
 20  20      287 non-null    int64
 21  21      287 non-null    int64
 22  22      287 non-null    int64
 23  23      287 non

In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin
class FirstCustomScaler(BaseEstimator, ClassifierMixin):

    def __init__(self, name='first_custom_scaler'):
        self.name = name
        self._fitted: bool = False
        self._scale: Optional[float] = None

    def fit(self, X: pd.DataFrame):
        scale = X.abs().max()
        scale = scale.replace(0, 1)
        self._scale = scale
        self._fitted = True

    def transform(self, X: pd.DataFrame, skip_first_step: bool = False):
        self._check_is_fitted()
        X = X.copy()

        if not skip_first_step:
          X = self._norm_by_columns(X)

        X = self._norm_by_rows(X)
        return X

    def _check_is_fitted(self):
        if not self._fitted:
            raise TypeError(
                "This instance is not fitted yet. Call 'fit' with "
                "appropriate arguments before using this estimator."
            )

    def _norm_by_columns(self, X):
        X = X / self._scale
        return X

    def _norm_by_rows(self, X):
        X = X.divide(
            np.sqrt(
                np.sum(
                    np.power(X, 2),
                    axis=1
                )
            ),
            axis='rows'
        )
        return X

In [3]:
from sklearn.base import BaseEstimator, ClassifierMixin
class SecondCustomScaler(BaseEstimator, ClassifierMixin):

    def __init__(self, name='second_custom_scaler') -> None:
        self.name = name
        self._fitted: bool = False
        self._scale: Optional[float] = None

    def fit(self, X: pd.DataFrame) -> None:
        scale = X.abs().max()
        scale = scale.replace(0, 1)
        self._scale = scale
        self._fitted = True

    def transform(self, X: pd.DataFrame, skip_first_step: bool = False):
        self._check_is_fitted()
        X = X.copy()

        X = self._norm_by_columns(X)

        norms = self._calculate_norms(X)
        X = self._divide_by_norms(X, norms)
        X['norm'] = norms

        norms = self._calculate_norms(X)
        X = self._divide_by_norms(X, norms)
        
        return X

    def _check_is_fitted(self):
        if not self._fitted:
            raise TypeError(
                "This instance is not fitted yet. Call 'fit' with "
                "appropriate arguments before using this estimator."
            )

    def _norm_by_columns(self, X):
        X = X / self._scale
        return X

    def _calculate_norms(self, X):
        return np.sqrt(
            np.sum(
                np.power(X, 2),
                axis=1
            )
        )
    
    def _divide_by_norms(self, X, norms):
      X = X.divide(norms, axis='rows')
      return X

    

In [6]:
t = pd.DataFrame([[1, 2], [10, 20], [3, 8]])

In [9]:
s = FirstCustomScaler()
s.fit(t)
res  = s.transform(t)
res

,0,1
0,0.707107,0.707107
1,0.707107,0.707107
2,0.600000,0.800000


In [10]:
s = SecondCustomScaler()
s.fit(t)
res  = s.transform(t)
res

,0,1,norm
0,0.700140,0.700140,0.140028
1,0.408248,0.408248,0.816497
2,0.536656,0.715542,0.447214


In [ ]:
#poly = PolynomialFeatures(2)
#train_X = poly.fit_transform(train_X)
#test_X = poly.fit_transform(test_X)

In [ ]:
s = SecondCustomScaler()
s.fit(train_X)
q  = s.transform(train_X)

In [ ]:
q[5]

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
282    0.0
283    0.0
284    0.0
285    0.0
286    0.0
Name: 5, Length: 287, dtype: float64

In [ ]:
clfs = {
    "Decision Tree Classifier": DecisionTreeClassifier(max_depth=None, min_samples_split=2, random_state=0),
    "Random Forest Classifier": RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),
    "Extra Trees Classifier": ExtraTreesClassifier(n_estimators=100, max_depth=None, min_samples_split=2, random_state=0),
    "Ada Boost Classifier": AdaBoostClassifier(base_estimator=None, n_estimators=100, learning_rate=1.0, algorithm='SAMME.R', random_state=None),
    "Gradient Boosting Classifier": GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0),
    "Bagging Classifier": BaggingClassifier(base_estimator=None, n_estimators=100, max_samples=1.0, max_features=1.0, bootstrap=True, bootstrap_features=False, oob_score=False, warm_start=False, n_jobs=None, random_state=None, verbose=0)
}

In [ ]:
scalers = {
    "First Custom Scaler": FirstCustomScaler,
    "Second Custom Scaler": SecondCustomScaler,
    "Max Abs Scaler": MaxAbsScaler,
    "Min Max Scaler": MinMaxScaler,
    "Standard Scaler": StandardScaler,
    "Robust Scaler": RobustScaler
}

In [ ]:
def train_and_test(clf, train_X, train_y, test_X, test_y):
  clf.fit(train_X, train_y)

  train_predictions = clf.predict(train_X)
  test_predictions = clf.predict(test_X)

  train_accuracy_score = accuracy_score(train_y, train_predictions)
  test_accuracy_score = accuracy_score(test_y, test_predictions)

  train_precision_score = precision_score(train_y, train_predictions, 
                                          average='macro'
                                          )
  test_precision_score = precision_score(test_y, test_predictions, 
                                         average='macro'
                                         )

  train_recall_score = recall_score(train_y, train_predictions, 
                                    average='macro'
                                    )
  test_recall_score = recall_score(test_y, test_predictions, 
                                   average='macro'
  )

  train_f1_score = f1_score(train_y, train_predictions, 
                            average='macro'
                            )
  test_f1_score = f1_score(test_y, test_predictions, 
                           average='macro'
                           )

  print("Train Accuracy :: " + str(train_accuracy_score))
  print("Train Precision :: " + str(train_precision_score))
  print("Train Recall :: " + str(train_recall_score))
  print("Train f1_score :: " + str(train_f1_score))

  print("Test Accuracy  :: " + str(test_accuracy_score))
  print("Test Precision :: " + str(test_precision_score))
  print("Test Recall :: " + str(test_recall_score))
  print("Test f1_score :: " + str(test_f1_score))

  return {
    "train_accuracy_score": train_accuracy_score,
    "test_accuracy_score": test_accuracy_score,
    "train_precision_score": train_precision_score,
    "test_precision_score": test_precision_score,
    "train_recall_score": train_recall_score,
    "test_recall_score": test_recall_score,
    "train_f1_score": train_f1_score,
    "test_f1_score": test_f1_score,
  }

In [ ]:
def run_for_all_scallers_and_classifier(train_X, train_y, test_X, test_y, dataset_name, save_to_excel: bool = True):
  result = defaultdict(dict)

  for scaler_name, scaler_cls in scalers.items():

    train_X_copy, train_y_copy, test_X_copy, test_y_copy = train_X.copy(), train_y.copy(), test_X.copy(), test_y.copy()

    scaler = scaler_cls()
    scaler.fit(train_X_copy)

    train_X_copy = scaler.transform(train_X_copy)
    test_X_copy = scaler.transform(test_X_copy)

    for classifier_name, classifier_instance in clfs.items():
      print(classifier_name)

      result[classifier_name][scaler_name] = train_and_test(classifier_instance, train_X_copy, train_y_copy, test_X_copy, test_y_copy)

  if save_to_excel:
    for clf_name, clf_content in result.items():
      res_df = pd.DataFrame(clf_content)
      res_df.to_excel(f'{dataset_name}_{clf_name}.xlsx')

    
    



In [ ]:
run_for_all_scallers_and_classifier(train_X, train_y, test_X, test_y, "dermatology")

Decision Tree Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.967741935483871
Test Precision :: 0.951388888888889
Test Recall :: 0.9513888888888888
Test f1_score :: 0.9488636363636364
Random Forest Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 1.0
Test Precision :: 1.0
Test Recall :: 1.0
Test f1_score :: 1.0
Extra Trees Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Ada Boost Classifier


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Accuracy :: 0.55
Train Precision :: 0.3855932203389831
Train Recall :: 0.5
Train f1_score :: 0.4129032258064516
Test Accuracy  :: 0.7258064516129032
Test Precision :: 0.40860215053763443
Test Recall :: 0.5
Test f1_score :: 0.43703703703703706
Gradient Boosting Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.967741935483871
Test Precision :: 0.9458333333333333
Test Recall :: 0.9458333333333333
Test f1_score :: 0.9458333333333334
Bagging Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Decision Tree Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9516129032258065
Test Precision :: 0.9351851851851851
Test Recall :: 0.9236111111111112
Test f1_score :: 0.922

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklea

Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 1.0
Test Precision :: 1.0
Test Recall :: 1.0
Test f1_score :: 1.0
Extra Trees Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Ada Boost Classifier


...

Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Bagging Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Decision Tree Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Random Forest Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Extra T

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Bagging Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Decision Tree Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Random Forest Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Extra T

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738
Bagging Classifier
Train Accuracy :: 1.0
Train Precision :: 1.0
Train Recall :: 1.0
Train f1_score :: 1.0
Test Accuracy  :: 0.9838709677419355
Test Precision :: 0.9722222222222223
Test Recall :: 0.9791666666666666
Test f1_score :: 0.9737373737373738


RangeIndex(start=0, stop=280, step=1)